So in this notebook I want to explore if we can predict some missing values with ML models. I dont see much potential in variables that have very strong inbalance such gender, credit history, education, self employed and so on. 

I will try to predict marriage status, loan amount, loan amount term

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv(r'data/train.csv')
data.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


## Prediction of marriage status

In [19]:
df = data.copy()
df.dropna(inplace=True)

# create some useful variables
df['TotalIncome'] = np.log(df['ApplicantIncome'] + df['CoapplicantIncome'])
df['ApplicantIncome'] = np.log(df['ApplicantIncome'])
df['loan-income_ratio'] = df['TotalIncome']/df['LoanAmount']

x = df.drop('Married', axis=1)
x = pd.get_dummies(x)

y = df['Married'].map({'No':0, 'Yes':1})

x.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,TotalIncome,loan-income_ratio,Loan_ID_LP001003,Loan_ID_LP001005,Loan_ID_LP001006,...,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_N,Loan_Status_Y
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,...,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,8.353961,1581.093583,144.735417,342.050000,0.854167,8.670081,0.077270,0.002083,0.002083,0.002083,...,0.085417,0.797917,0.202083,0.862500,0.137500,0.289583,0.397917,0.312500,0.308333,0.691667
std,0.621877,2617.692267,80.508164,65.212401,0.353307,0.536939,0.056802,0.045644,0.045644,0.045644,...,0.279792,0.401973,0.401973,0.344734,0.344734,0.454042,0.489979,0.463996,0.462287,0.462287
min,5.010635,0.000000,9.000000,36.000000,0.000000,7.273786,0.016462,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.972035,0.000000,100.000000,360.000000,1.000000,8.330379,0.051393,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.258163,1084.500000,128.000000,360.000000,1.000000,8.598312,0.067878,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,8.674600,2253.250000,170.000000,360.000000,1.000000,8.945394,0.082804,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,11.302204,33837.000000,600.000000,480.000000,1.000000,11.302204,0.863779,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
from sklearn.model_selection import train_test_split
x_tr, x_te, y_tr, y_te = train_test_split(x, y, test_size=0.15, random_state=5)

In [58]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500, max_depth=4, class_weight='balanced')
model.fit(x_tr, y_tr)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [59]:
from sklearn.metrics import f1_score, accuracy_score
pred = model.predict(x_te)

print('Training F1: ',
      f1_score(y_tr, model.predict(x_tr)),
      accuracy_score(y_tr, model.predict(x_tr))
      )

print('\n Test F1: ',
      f1_score(y_te, pred),
      accuracy_score(y_te, pred)
     )

Training F1:  0.8588007736943907 0.821078431372549

 Test F1:  0.8775510204081632 0.8333333333333334


If we fill NA with majority class we are likely to hit about 60% accuracy so 83% accuracy on test set is great result!

## Loan Amount Term

In [60]:
df = data.copy()
df.dropna(inplace=True)

# create some useful variables
df['TotalIncome'] = np.log(df['ApplicantIncome'] + df['CoapplicantIncome'])
df['ApplicantIncome'] = np.log(df['ApplicantIncome'])
df['loan-income_ratio'] = df['TotalIncome']/df['LoanAmount']

x = df.drop('Loan_Amount_Term', axis=1)
x = pd.get_dummies(x)

y = df['Loan_Amount_Term']

In [94]:
x_tr, x_te, y_tr, y_te = train_test_split(x, y, test_size=0.15, random_state=5)
import xgboost as xgb
model = xgb.XGBClassifier(n_estimators=500, max_depth=6, reg_lambda=4.5,
                          reg_alpha=0.3, class_weight='balanced', n_jobs=-1)
#model = RandomForestClassifier(n_estimators=500, max_depth=8)
model.fit(x_tr, y_tr)

XGBClassifier(base_score=0.5, booster='gbtree', class_weight='balanced',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0.3, reg_lambda=4.5, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [95]:
from sklearn.metrics import mean_absolute_error
pred = model.predict(x_te)

# I will use here some unusuall combination of metrics because target variable is numeric but descrete
print('Training F1, MAE: ',
      f1_score(y_tr, model.predict(x_tr), average='weighted'),
      mean_absolute_error(y_tr, model.predict(x_tr)))

print('\n Test F1, MAE: ',
      f1_score(y_te, pred, average='weighted'),
      mean_absolute_error(y_te, pred))

Training F1, MAE:  1.0 0.0

 Test F1, MAE:  0.7515432098765432 25.833333333333332


Here we overfit but its something to work with into pipeline. XGB outperformed Random Forest by ~ 3 dollars in MAE but Random Forest didnt overfit.

Here are results from RF:

Training F1, MAE:  0.807, 22.59

Test F1, MAE:  0.719, 28.33

## Loan Amount

In [96]:
df = data.copy()
df.dropna(inplace=True)

# create some useful variables
df['TotalIncome'] = np.log(df['ApplicantIncome'] + df['CoapplicantIncome'])
df['ApplicantIncome'] = np.log(df['ApplicantIncome'])
df['loan-income_ratio'] = df['TotalIncome']/df['LoanAmount']

x = df.drop('LoanAmount', axis=1)
x = pd.get_dummies(x)

y = df['LoanAmount']

In [97]:
from sklearn.ensemble import RandomForestRegressor
x_tr, x_te, y_tr, y_te = train_test_split(x, y, test_size=0.15, random_state=5)
#model = xgb.XGBClassifier(n_estimators=500, max_depth=6, reg_lambda=4.5,
#                          reg_alpha=0.3, n_jobs=-1)
model = RandomForestRegressor(n_estimators=300, max_depth=6)
model.fit(x_tr, y_tr)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [99]:
from sklearn.metrics import r2_score
pred = model.predict(x_te)

print('Training R^2, MAE: ',
      r2_score(y_tr, model.predict(x_tr)),
      mean_absolute_error(y_tr, model.predict(x_tr)))

print('\n Test R^2, MAE: ',
      r2_score(y_te, pred),
      mean_absolute_error(y_te, pred))

Training R^2, MAE:  0.9973450973537658 2.4509948777554076

 Test R^2, MAE:  0.9944683806760204 3.552533684410802


Loan Amount StDev is about 80 and our model can explain about 99% of variance, these results are great!